In [1]:
from mujoco_parser import MuJoCoParserClass
import cv2,os,mujoco,mujoco_viewer
import time
import numpy as np

scene_path = '../../asset/mujoco/ur5e/scene_ur5e_test.xml'
env = MuJoCoParserClass(name='UR5e',rel_xml_path=scene_path,VERBOSE=True)



dt:[0.0020] HZ:[500]
n_dof (=nv):[6]
n_geom:[33]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
n_body:[10]
body_names:['world', 'base', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'eef_tool', 'ellipsoid_body']
n_joint:[6]
joint_names:['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
joint_types:[3 3 3 3 3 3]
joint_ranges:
[[-6.28319  6.28319]
 [-6.28319  6.28319]
 [-3.1415   3.1415 ]
 [-6.28319  6.28319]
 [-6.28319  6.28319]
 [-6.28319  6.28319]]
n_rev_joint:[6]
rev_joint_idxs:[0 1 2 3 4 5]
rev_joint_names:['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
rev_joint_mins:[-6.28319 -6.28319 -3.1415  -6.28319 -6.28319 -6.28319]
rev_joint_maxs:[6.283

In [2]:
import scipy
import scipy.linalg

class Hybrid_Controller(object):
    def __init__(self,
                 dt=0.002,
                 dim=6,
                 b_T_c = np.eye(4),
                 flat_top = True,
                 max_vel=5,
                ):
        
        self.dt = dt
        self.dim = dim
        self.b_T_c = b_T_c
        self.flattop = flat_top
        self.max_vel = max_vel
        self.emergency_stop = False
        self.integral = np.zeros((dim-1,1))
        self.nu_v_old = np.zeros((dim-1,1))
        
    def update(self, pose, h_d, h_e, v_d, v_e):
        pose = np.array(pose).reshape(6,1)
        h_d = np.array(h_d).reshape(6,1)
        h_e = np.array(h_e).reshape(6,1)
        v_d = np.array(v_d).reshape(6,1)
        v_e = np.array(v_e).reshape(6,1)

        v_ref = self.get_control_input(pose, h_d, h_e, v_d, v_e)
        return np.transpose(v_ref)
    
    def get_control_input(self, pose, h_d, h_e, v_d, v_e):
        n = self.get_normal(pose[0:3])
        Sf = self.get_Sf(n)
        Sv = self.get_Sv(n)

        Sf_pinv = np.linalg.pinv(Sf)
        Sv_pinv = np.linalg.pinv(Sv)

        lambda_d = Sf_pinv @ h_d
        lambda_e = Sf_pinv @ h_e


        lambda_v = self.get_force_control(lambda_d, lambda_e)

        nu_d = Sv_pinv @ v_d
        nu_e = Sv_pinv @ v_e

        #nu_v = Sv_pinv @ v_d
        nu_v = self.get_velocity_control(nu_d, nu_e)

        # return Sv @ (Sv_pinv @ v_d)
        #return Sv @ nu_v
        return Sf * lambda_v + Sv @ nu_v

    def get_normal(self, xpos):

        b_p = np.ones((4,1))
        b_p[0:3] = xpos

        c_p = np.linalg.inv(self.b_T_c) @ b_p

        x = c_p[0]
        y = c_p[1]

        if(self.flattop):
            c_n = np.array([[0], [0], [1]])
        else:
            c_n = np.array([[x/np.sqrt(x*x + y*y)], [y/np.sqrt(x*x + y*y)], [0]])

        return c_n
    
    def get_Sf(self, n):
        zero = np.zeros((3,1))
        Sf = np.vstack((n, zero))

        return Sf
    
    def get_Sv(self, n):
        Sv = np.zeros((6,5))
        Q = np.array(scipy.linalg.qr(n)[0])
        a = []
        # print(n)
        # print(n_np)
        for i in range(3):
            if (np.allclose(Q[0:3,i],np.array(n).T)) or np.allclose(Q[0:3,i],-np.array(n).T):
                c = Q[0:3,i]
            elif len(a)==0:
                a = Q[0:3,i]
            else:
                b = Q[0:3,i]
                
        if(np.allclose(np.cross(a,b),c)):
            Sv[0:3, 0] = a
            Sv[0:3, 1] = b
        else:
            Sv[0:3, 0] = b
            Sv[0:3, 1] = a

        Sv[3:6, 2:5] = np.eye(3)

        return Sv

    
    def get_force_control(self, lambda_d, lambda_e):
        K_P = 20
        K_V = 0.1
        max_force = 10
        no_contact_speed = 0.5

        # Emergency stop if force is too big
        if(lambda_e > max_force):
            print("Max force: Emergency stop")
            self.emergency_stop = True

        #If no contact
        #if lambda_e < 0.01:
        #    # print("No contact")
        #    return no_contact_speed

        error = lambda_d - lambda_e

        lambda_v = lambda_d + K_P * error - K_V * lambda_e

        # # Output speed clamping to 
        # if(np.linalg.norm(lambda_v) > max_force):
        #     lambda_v = (lambda_v / np.linalg.norm(lambda_v)) * max_force

        if lambda_v < 0:
            print("Negative")

        return lambda_v
    
    def get_velocity_control(self, nu_d, nu_e):
                
        K_P = 0.305
        K_I = 0.1423
        max_speed = 5.0
        max_acc = 0.03
        out_alpha = 0.1

        error = nu_d - nu_e

        proportional = K_P * error

        #Anti windup
        if (np.linalg.norm(nu_e) < max_speed):
            self.integral += K_I * error * self.dt

        nu_v = nu_d + proportional + self.integral

        nu_v_norm = np.linalg.norm(nu_v)
        #max speed clamping
        if (np.linalg.norm(nu_v) > max_speed):
            nu_v = (nu_v/nu_v_norm) * max_speed
            #print("Max speed")

        #acceleration clamping
        if np.linalg.norm(nu_v - self.nu_v_old ) > max_acc:
            nu_v = self.nu_v_old  + ((nu_v - self.nu_v_old ) / np.linalg.norm(nu_v - self.nu_v_old )) * max_acc
        
        # if np.linalg.norm(nu_v) > np.linalg.norm(nu_d):
        #     nu_v = (nu_v / np.linalg.norm(nu_v)) * np.linalg.norm(nu_d)

        # Output smoothing using EMA
        nu_v_smoothed = out_alpha * self.nu_v_old + (1.0 - out_alpha) * nu_v
        self.nu_v_old = nu_v_smoothed

        return nu_v_smoothed

def desired_speed(T, t):
    r = 0.2
    omega = 2*np.pi/T

    dx = -np.sin(omega*t - np.pi/2)*omega*r
    dy = np.cos(omega*t - np.pi/2)*omega*r

    #return [0,0,0,0,0,0]
    return [dx,dy,0,0,0,0]

def desired_force():
    return [0,0,0,0,0,0]

In [3]:
init_q = [-1.03631, -0.740625, 1.20066, -1.78359, -1.71383, -0.00717087]
env.set_initial_qpos(init_q)
env.run_sim()

: 